<a href="https://colab.research.google.com/github/YolandaMDavis/NSSADNN_IQA/blob/wildtrack-iqa/generate_image_feature_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is used to create a non-reference iqa data set from the images provided by Wildtrack. Taking a directory of images, the tool will calculate all IQA measure for each imaage and store them in files (presplit for training validation and test) along with image metadata.

In [1]:
#Only needed to copy data to local drive can be skipped if zip file is already available in working folder
import shutil

# mount google drive
from google.colab import drive
drive.mount('/content/drive') # for google colab. adjust accordingly
PARENT_DIR = '/content/drive/MyDrive/Wildtrack Group/IQA' 

# copy and extract tar file
shutil.copy(PARENT_DIR + '/data/WildTrack_Raw.zip', 'WildTrack_Raw.zip')

Mounted at /content/drive


'WildTrack_Raw.zip'

In [2]:
# clone the repository and move data file
!git clone https://github.com/YolandaMDavis/NSSADNN_IQA.git
!mv WildTrack_Raw.zip NSSADNN_IQA/.
%cd "NSSADNN_IQA"
!git checkout wildtrack-iqa

Cloning into 'NSSADNN_IQA'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 90 (delta 42), reused 36 (delta 10), pack-reused 0
Unpacking objects: 100% (90/90), done.
/content/NSSADNN_IQA
Branch 'wildtrack-iqa' set up to track remote branch 'wildtrack-iqa' from 'origin'.
Switched to a new branch 'wildtrack-iqa'


In [3]:
from zipfile import ZipFile
# extract zip file
with ZipFile('WildTrack_Raw.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [4]:
# set directory variables and 
import os

import numpy as np
from scipy import stats
import torch.nn as nn
import random

root_dir = '/content/NSSADNN_IQA'
data_dir = root_dir + '/RAW'
image_reference_file_suffix = '_image_references.csv'

%cd "/content/NSSADNN_IQA"

/content/NSSADNN_IQA


In [26]:
from torch.utils import data
import csv
import random
import torch
import yaml
import cv2
from brisque import brisque
from niqe import niqe
from piqe import piqe

from torch.utils.data import Dataset


def get_image_row(image_reference):
    file_path = os.path.join(image_reference[0], image_reference[3])
    print("Processing image for features:" + file_path)

    im_features = cv2.imread(file_path)
    im_features = cv2.cvtColor(im_features, cv2.COLOR_BGR2RGB)
    brisque_features = brisque(im_features)
    n_score = niqe(im_features)
    p_score,_,_,_ = piqe(im_features)
    other_scores = [n_score, p_score]
    iqa_features = np.append(brisque_features,np.array(other_scores))
    iqa_features = tuple(iqa_features.tolist())
    return image_reference + iqa_features

def generate_file(dataset):
  ref_file_name = root_dir + '/' + dataset[0] + image_reference_file_suffix
  with open(ref_file_name, 'w', newline='') as csvfile:
      image_ref_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
      header_columns = ['image_location','species','animal_class','image_name','subj_score']
      brisque_columns = ['feature_' + str(i) for i in range(0,36)]
      header_columns+=brisque_columns
      header_columns+=['n_score','p_score']
      image_ref_writer.writerow(header_columns)
      image_ref_writer.writerows(dataset[1])

def generate_data_files(sample_percentage=1):

    image_reference_list = []

    subdirectories = list(os.walk(data_dir, topdown=False))[:-1]
    for subdir in subdirectories:
        image_location = subdir[0]
        images = subdir[2]
        species_rating = image_location.rsplit('/', 1)[-1].replace('_', ' ')
        score = int(species_rating.rsplit(' ', 1)[-1])
        species_class = species_rating.rsplit(' ', 1)[:-1][0]
        if len(species_class.rsplit(' ', 1)) > 1:
            species = species_class.rsplit(' ')[0]
            animal_class = ' '.join(species_class.rsplit(' ')[1:])
        else:
            animal_class = 'Unknown'
            species = species_class

        for image in images:
            image_reference = (image_location, species, animal_class, image, score)
            image_reference_list.append(image_reference)
        

    # shuffle then split
    seed = 1234
    random.Random(seed).shuffle(image_reference_list)

    train_index = int(len(image_reference_list) *0.6 *sample_percentage)
    test_index = -int(len(image_reference_list) * 0.2 * sample_percentage) 
    val_index = test_index * 2

    training = image_reference_list[:train_index]
    validation = image_reference_list[val_index:test_index]
    testing = image_reference_list[test_index:]

    training_features = [get_image_row(image_reference) for image_reference in training]
    validation_features = [get_image_row(image_reference) for image_reference in validation]
    test_features = [get_image_row(image_reference) for image_reference in testing]

    # generated reference splits
    for dataset in [('training', training_features), ('validation', validation_features), ('testing', test_features)]:
        dataset_type = dataset[0]
        print("Processing File: " + dataset_type)
        generate_file(dataset)
        shutil.copy(dataset_type + '_image_references.csv', PARENT_DIR + '/data/' + dataset_type + '_image_references.csv') 

In [27]:
# take a percentage of the full data set as a training/test/validation sample
sample_size=.1
generate_data_files(sample_size)

Processing image for features:/content/NSSADNN_IQA/RAW/Otter_Eurasian_4/254d47305e5611ec81ca0242ac100102.jpg
Processing image for features:/content/NSSADNN_IQA/RAW/Leopard_African_4/99929e22540411ebb49800155d5f029b.jpg
Processing image for features:/content/NSSADNN_IQA/RAW/Tapir_Lowland_4/72d025e832a311ebbaf80242ac110002.jpg
Processing image for features:/content/NSSADNN_IQA/RAW/Tapir_Lowland_4/82d7d00832a311ebbaf80242ac110002.jpg
Processing image for features:/content/NSSADNN_IQA/RAW/Rhino_White_5/db7cffa0d3dd11ea91590242ac1c0002.jpg
Processing image for features:/content/NSSADNN_IQA/RAW/Otter_Eurasian_4/c5cd5096d6ae11eab34b0242ac110002.jpg
Processing image for features:/content/NSSADNN_IQA/RAW/Tiger_Bengal_5/10f3c1aad3de11ea91590242ac1c0002.jpg
Processing image for features:/content/NSSADNN_IQA/RAW/Rhino_Black_4/24d15f6cc0ff11ea82a50242ac1c0002.jpg
Processing image for features:/content/NSSADNN_IQA/RAW/Otter_Eurasian_4/f7672c72358011eb987f00155d48c714.jpg
Processing image for feature

In [25]:
shutil.copy('training_image_references.csv', PARENT_DIR + '/data/' + 'training' + '_image_references.csv') 

'/content/drive/MyDrive/Wildtrack Group/IQA/data/training_image_references.csv'